In [70]:
import pandas as pd
from IPython.display import HTML, clear_output


df = pd.read_json('regen.json')
# if all(item in df.columns for item in ['curation_status', 'curation_message']):
#   df.fillna(value={'curation_status': 0, 'curation_message': ''}, inplace=True)
# else:
df['curation_status'] = 0
df['curation_message'] = ''
df.tail(3)

,instruction,output,most_similar_instructions,avg_similarity_score,curation_status,curation_message
28,Buatlah website toko online yang menjual produ...,"<!DOCTYPE html>\n<html lang=""id"">\n\n<head>\n ...","{'Buatlah website toko online ""Fashion Avenue""...",0.298571,0,
29,Buatlah website toko online yang menjual produ...,"<!DOCTYPE html>\n<html lang=""id"">\n\n<head>\n ...",{'Buatlah website toko online yang menjual pro...,0.306212,0,
30,"Buatlah website restoran dengan judul ""Culinar...","<!DOCTYPE html>\n<html lang=""id"">\n\n<head>\n ...","{'Buatlah website blog dengan judul ""Ruang Ins...",0.317654,0,


In [71]:
curation_df = pd.DataFrame(columns=['curation_status', 'curation_message'])

In [72]:
for idx, row in df[df['curation_status'] == 0].iterrows():
  last_idx = idx
  instruction, output, most_similar_instructions, avg_similarity_score, _, __ = row
  stop = False
  while not stop:
    clear_output(wait=True)
    display(HTML(output))
    
    print(f'''Instruction: {instruction}
    Output: {output}''')
    
    curator_input = input(f'{idx}. Is the data good? (yes/no/mark/stop):')
    curator_input = curator_input.strip()
    if curator_input == 'yes':
      curation_df.loc[idx] = {
        'curation_status': 1,
        'curation_message': '',
      }
      break
    elif curator_input == 'no':
      curation_df.loc[idx] = {
        'curation_status': -1,
        'curation_message': '',
      }
      break
    # Mark the data if you think the data can be fixed/improved manually
    elif curator_input == 'mark':
      suggestion = input('Suggestion: ')
      curation_df.loc[idx] = {
        'curation_status': 0,
        'curation_message': suggestion,
      }
      break
    elif curator_input == 'stop':
      stop = True
    else:
      print('Invalid input:', '"' + curator_input + '"')
  if stop:
    break
clear_output()

In [73]:
curation_df

,curation_status,curation_message
0,1,
1,1,
2,1,
3,1,
4,1,
5,-1,
6,1,
7,1,
8,1,
9,1,


In [74]:
curated_df = curation_df.combine_first(df)[df.columns]
curated_df.iloc[curation_df.index]

,instruction,output,most_similar_instructions,avg_similarity_score,curation_status,curation_message
0,Buatlah website blog yang menampilkan artikel-...,"<!DOCTYPE html>\n<html lang=""id"">\n\n<head>\n ...","{'Buatlah website e-commerce ""Toko Saya"" yang ...",0.308279,1,
1,"Buatlah website ""Reservasi Hotel"" yang memungk...","<!DOCTYPE html>\n<html lang=""id"">\n\n<head>\n ...","{'Buatlah website e-commerce ""Toko Furnitur"" y...",0.275232,1,
2,"Buatlah sebuah website berita ""Berita Hari Ini...","<!DOCTYPE html>\n<html lang=""id"">\n\n<head>\n ...",{'Buatlah website portofolio yang menampilkan ...,0.292180,1,
3,"Buatlah website berita ""Berita Terkini"" yang m...","<!DOCTYPE html>\n<html lang=""id"">\n\n<head>\n ...","{'Buatlah website e-commerce ""Toko Furnitur"" y...",0.273642,1,
4,"Buatlah website blog dengan judul ""Ruang Inspi...","<!DOCTYPE html>\n<html lang=""id"">\n\n<head>\n ...",{'Buatlah website blog yang menampilkan artike...,0.307253,1,
5,Buatlah website untuk sebuah restoran yang men...,"<!DOCTYPE html>\n<html lang=""id"">\n\n<head>\n ...",{'Buatlah website bisnis untuk perusahaan bern...,0.272819,-1,
6,Buatlah website tutorial yang menampilkan daft...,"<!DOCTYPE html>\n<html lang=""id"">\n\n<head>\n ...","{'Buatlah website blog dengan judul ""Ruang Ins...",0.305832,1,
7,"Buatlah website ""Agensi Pariwisata"" yang menam...","<!DOCTYPE html>\n<html lang=""id"">\n\n<head>\n ...","{'Buatlah website ""Reservasi Hotel"" yang memun...",0.290896,1,
8,"Buatlah website e-commerce dengan nama ""Toko O...","<!DOCTYPE html>\n<html lang=""id"">\n\n<head>\n ...",{'Buatlah website blog yang berisi artikel-art...,0.238779,1,
9,Buatlah website informasi tentang destinasi wi...,"<!DOCTYPE html>\n<html lang=""id"">\n\n<head>\n ...","{'Buatlah website e-commerce dengan nama ""Toko...",0.245673,1,


In [75]:
curated_df[curated_df['curation_status'] != -1].to_json('regen-curated.json', orient="records")
print('regen-curated.json exported sucessfully!')

regen-curated.json exported sucessfully!


In [3]:
ok_df = pd.read_json('regen.json')
ok_df[['instruction', 'output']].to_json('webgen-dataset-2.json', orient='records')
print('webgen-dataset-2.json exported sucessfully!')

webgen-dataset-2.json exported sucessfully!
